# TME 2-3 : Diffusion


In [12]:
# Parsing
with open("data/cascades_train.txt") as f:
    cascades_train = f.read()

diffusions_train = []
times_train = []
for line in cascades_train.split('\n'):
    diff = line.strip().split(';')
    diff = [s.split(':') for s in diff if len(s) > 1]
    diff = {int(l[0]):float(l[1]) for l in diff}
    times = {}
    for elt, t in diff.items():
        if t in times:
            times[t].append(elt)
        else:
            times[t] = [elt]
    diffusions_train.append(diff)
    times_train.append(times)

In [21]:
from functools import reduce
# Dans 'diffusions_train' chaque élément est une diffusion,
# chaque diffusion est un dictionnaire {élément:temps d'infection}
print("Diffusion n°0 :", diffusions_train[0])

# Dans 'times_train', chaque élément est une diffusion,
# chaque diffusion est un dictionnaire {temps:[éléments infectés]}
print("Diffusion n°0 :", times_train[0])

all_times = reduce(lambda l,m:set(l)|set(m), [d.keys() for d in times_train])
print("Tous les temps :", all_times)
all_nodes = reduce(lambda l,m:set(l)|set(m), [d.keys() for d in diffusions_train])
print("Tous les noeuds :", all_nodes)

Diffusion n°0 : {3: 4.0, 4: 10.0, 5: 8.0, 8: 3.0, 73: 4.0, 12: 7.0, 98: 5.0, 82: 11.0, 84: 5.0, 86: 5.0, 89: 3.0, 26: 1.0, 93: 2.0, 96: 7.0, 34: 9.0, 41: 2.0, 42: 2.0, 43: 4.0, 47: 6.0, 48: 6.0, 50: 7.0, 52: 8.0, 54: 9.0, 55: 7.0, 56: 1.0, 20: 7.0, 61: 5.0}
Diffusion n°0 : {1.0: [26, 56], 2.0: [93, 41, 42], 3.0: [8, 89], 4.0: [3, 73, 43], 5.0: [98, 84, 86, 61], 6.0: [47, 48], 7.0: [12, 96, 50, 55, 20], 8.0: [5, 52], 9.0: [34, 54], 10.0: [4], 11.0: [82]}
Tous les temps : {1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0}
Tous les noeuds : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96

In [30]:
# Apprentissage

# Calcul de D+ et D-
Dplus = []
Dminus = []
for u in all_nodes:
    Dplus.append([])
    Dminus.append([])
    for v in all_nodes:
        Dplus[u].append(set())
        Dminus[u].append(0)
        for idx, diff in enumerate(diffusions_train):
            if v in diff and u in diff and diff[v] > diff[u]:
                Dplus[u][v].add(idx)
            if v not in diff and u in diff:
                Dminus[u][v] += 1
                


In [31]:
print(Dplus[5][6])
print(Dminus[5][6])

{2944, 4225, 390, 2438, 3591, 2825, 4746, 4107, 2316, 4495, 3986, 1428, 1301, 151, 2202, 3098, 2076, 3614, 1056, 1825, 546, 675, 2722, 3623, 3496, 1194, 307, 3380, 309, 4915, 2615, 4023, 441, 827, 1341, 1598, 4292, 3653, 4165, 967, 2887, 4937, 2764, 4174, 4688, 1105, 1107, 852, 1621, 2646, 3540, 984, 3033, 2266, 2650, 2140, 3798, 4061, 3557, 4325, 999, 2407, 362, 746, 1130, 1902, 3823, 1904, 2672, 4079, 4220, 1782, 3321, 3963, 124, 1277}
531


In [34]:
import numpy as np

def compute_P(diff, times, v, theta):
    """ Calcule P_{tdv}(v)
    :param times: dict {temps:[éléments]}"""
    produit = 1
    t_vD = diff[v]
    if t_vD - 1 in times:
        for w in times[t_vD - 1]:
            produit *= (1 - theta[w,v])
    return 1 - produit

old_theta_hat = np.zeros((len(all_nodes), len(all_nodes)))
new_theta_hat = np.random.random((len(all_nodes), len(all_nodes)))

it = 0
while not np.allclose(old_theta_hat, new_theta_hat) and it < 500:
    print("==== %d ====" % it)
    theta_star = np.zeros_like(new_theta_hat)
    for u in all_nodes:
        for v in all_nodes:
            theta_star[u,v] = sum([1/compute_P(diff, times, v, new_theta_hat) 
                                    for diff, times in zip(diffusions_train, times_train)
                                     if v in diff and #todo check
                                  ])
            theta_star[u,v] *= new_theta_hat[u,v] / (len(Dplus[u][v]) + Dminus[u][v])
    old_theta_hat = new_theta_hat
    new_theta_hat = theta_star
    it += 1
    
print(new_theta_hat)

==== 0 ====


ZeroDivisionError: division by zero